In [ ]:
import os, tensorflow, keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization, Dropout, Lambda, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load datasets

In [ ]:
# load the dataset => training, cross-validation and test sets

train_dir = os.path.join("/kaggle/input/isic-2017-dataset/content/Linear_Exact_Aug/Train")
test_dir = os.path.join("/kaggle/input/isic-2017-dataset/content/Linear_Exact_Aug/Test")
valid_dir = os.path.join("/kaggle/input/isic-2017-dataset/content/Linear_Exact_Aug/Valid")

# print("Train Directory:", train_dir)
# print("Test Directory:", test_dir)
# print("Validation Directory:", valid_dir)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# CNN Model (DenseNet architecture)

In [ ]:
# Load VGG19 without the top layer
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = Lambda(lambda z: -z)(base_model.output) 
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x) 
x = Lambda(lambda z: -z)(x)  

x = Lambda(lambda z: -z)(base_model.output) 
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x) 
x = Lambda(lambda z: -z)(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(negative_slope=0.02)(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(32, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(negative_slope=0.02)(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

# Output Layer
x = Dense(3, activation='softmax', kernel_regularizer=l2(0.01))(x)

denseNetLeakyV2Model = Model(inputs=base_model.input, outputs=x)

## Model Summary

In [ ]:
denseNetLeakyV2Model.summary()

# Model compilation

In [ ]:
# compiling the model
denseNetLeakyV2Model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler: Reduce LR if validation loss stops improving
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

# setting checkpoint for the optimal weights corresponding to the minimum loss
checkpoint = ModelCheckpoint("/kaggle/working/denseNetLeakyV2Model.weights.h5", monitor="val_loss", save_best_only=True, save_weights_only=True, mode="min", verbose=1)

# Early Stopping: Stop training if val_loss doesn't improve for 5 epochs
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
denseNetLeakyV2Model.load_weights("/kaggle/input/leakyrelu-densenet-weights/denseNetLeakyV2Model.weights.h5")

# Model fitting

In [ ]:
# fitting the model
"""history = denseNetLeakyV2Model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs = 30,
    callbacks=[checkpoint, lr_scheduler]
)""

# Loading optimized weights

In [ ]:
# Load and display the optimized weights

# Load best weights
denseNetLeakyV2Model.save_weights("/kaggle/working/denseNetLeakyV2Model.weights.h5")

# Get all layer weights as a list of NumPy arrays
optimized_weights = denseNetLeakyV2Model.get_weights()

# Print weight values for each layer
'''for i, weight in enumerate(optimized_weights):
    print(f"Weight {i+1}: Shape = {weight.shape}")
    print(weight)
    print("\n" + "-"*50)***
'''

# Validation dataset accuracy

In [ ]:
# validation loss and validation accurracy
from sklearn.metrics import accuracy_score
import numpy as np

class_map = {0: "melanoma", 1: "nevus", 2:"seborrheic-keratosis"}

# Model evaluation
validation_loss, validation_accuracy = denseNetLeakyV2Model.evaluate(valid_generator)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")
print(f"Validation loss: {validation_loss:.4f}")


# Model predictions
valid_generator.shuffle = True
predictions = denseNetLeakyV2Model.predict(valid_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = valid_generator.classes
test_length = len(true_classes)

# Print predicted classes on validation set
print("Predicted vs True Classes:")
for i in range(test_length):
    print(f"Sample {i+1:<3}: Predicted = {class_map[predicted_classes[i]]:<25} True = {class_map[true_classes[i]]:<25}")

# individual percentage of correctly predicted classes
mel, nev, seb = 0, 0, 0
for i in range(test_length):
    mel += (true_classes[i] == 0)
    nev += (true_classes[i] == 1)
    seb += (true_classes[i] == 2)

print(mel, nev, seb)

correct_mel, correct_nev, correct_seb = 0, 0, 0

for i in range(mel):
    correct_mel += (predicted_classes[i] == 0)

for i in range(mel, mel+nev):
    correct_nev += (predicted_classes[i] == 1)

for i in range(mel+nev, test_length):
    correct_seb += (predicted_classes[i] == 2)

print("Correctly predicted melanoma: ", (correct_mel / mel) * 100, "%")
print("Correctly predicted nevus: ", (correct_nev / nev) * 100, "%")
print("Correctly predicted seborrheic-keratosis: ", (correct_seb / seb) * 100, "%")


# Computing correct prediction percentage
accuracy = accuracy_score(true_classes, predicted_classes) * 100
print(f"Correct Prediction Percentage: {accuracy:.2f}%")

# test dataset accuracy

In [ ]:
# test loss and test accurracy
from sklearn.metrics import accuracy_score
import numpy as np

class_map = {0: "melanoma", 1: "nevus", 2:"seborrheic-keratosis"}

# Model evaluation
test_loss, test_accuracy = denseNetLeakyV2Model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test loss: {test_loss:.4f}")


# Model predictions
test_generator.shuffle = True
predictions = denseNetLeakyV2Model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
test_length = len(true_classes)

# Print predicted classes on the test dataset
print("Predicted vs True Classes:")
for i in range(test_length):
    print(f"Sample {i+1:<3}: Predicted = {class_map[predicted_classes[i]]:<25} True = {class_map[true_classes[i]]:<25}")

# individual percentage of correctly predicted classes
mel, nev, seb = 0, 0, 0
for i in range(test_length):
    mel += (true_classes[i] == 0)
    nev += (true_classes[i] == 1)
    seb += (true_classes[i] == 2)

print(mel, nev, seb)

correct_mel, correct_nev, correct_seb = 0, 0, 0

for i in range(mel):
    correct_mel += (predicted_classes[i] == 0)

for i in range(mel, mel+nev):
    correct_nev += (predicted_classes[i] == 1)

for i in range(mel+nev, test_length):
    correct_seb += (predicted_classes[i] == 2)

print("Correctly predicted melanoma: ", (correct_mel / mel) * 100, "%")
print("Correctly predicted nevus: ", (correct_nev / nev) * 100, "%")
print("Correctly predicted seborrheic-keratosis: ", (correct_seb / seb) * 100, "%")


# Computing correct prediction percentage
accuracy = accuracy_score(true_classes, predicted_classes) * 100
print(f"Correct Prediction Percentage: {accuracy:.2f}%")

# Scatterplot with classifier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC  # Using SVM for decision boundaries

# Apply PCA to reduce dimensions to 2D
pca = PCA(n_components=2)
test_data_2d = pca.fit_transform(predictions)

# Train a classifier (SVM) on the PCA-transformed data
svm_model = SVC(kernel="linear", C=1.0)  # Linear SVM for decision boundaries
svm_model.fit(test_data_2d, predicted_classes)

# Create a mesh grid for shading the decision regions
x_min, x_max = test_data_2d[:, 0].min() - 0.5, test_data_2d[:, 0].max() + 0.5
y_min, y_max = test_data_2d[:, 1].min() - 0.5, test_data_2d[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

# Predict class for each point in the mesh grid
Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Define color mappings
colors = ['red', 'blue', 'green']
class_names = ['Melanoma', 'Nevus', 'Seborrheic Keratosis']
region_colors = ["red", "blue", "green"]

# Plot decision boundary with shading
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)

# Scatter plot with actual test points
sns.scatterplot(x=test_data_2d[:, 0], y=test_data_2d[:, 1], hue=predicted_classes, palette=colors, edgecolor='k', s=50)

# Improve visualization
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Decision Boundaries for Skin Cancer Classification")
plt.legend(handles=[plt.Line2D([0], [0], marker="o", color="w", markerfacecolor=colors[i], markersize=10, label=class_names[i]) for i in range(3)], title="Skin Cancer Type", loc="upper left")
plt.xlim(-0.5, 0.85)  
plt.ylim(-0.9, 0.9)   


plt.grid(True)
plt.show()

# Training vs validation accuracy

In [ ]:
# Get accuracy and loss values from the training history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot Training Accuracy vs. Validation Accuracy
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_accuracy, label='Training Accuracy', linestyle='-')
plt.plot(val_accuracy, label='Validation Accuracy', linestyle='-')
plt.title('Training Accuracy vs. Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

# Training vs validation loss

In [ ]:
# Get accuracy and loss values from the training history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot Training Loss vs. Validation Loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss', linestyle='-')
plt.plot(val_loss, label='Validation Loss', linestyle='-')
plt.title('Training Loss vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(true_classes, predicted_classes)
sns.heatmap(cm, annot=True, cmap="Blues", fmt='d', xticklabels=class_map.values(), yticklabels=class_map.values())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()